In [1]:
# === SEKCJA 1: IMPORT I KONFIGURACJA ===
import pandas as pd
import numpy as np
import mlflow
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, pull
from IPython.display import display

# Konfiguracja MLflow
MLFLOW_EXPERIMENT_NAME = 'Warsaw_Street_Prediction_v3_FINAL'
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)
print(f"MLflow ustawiony. Eksperyment: '{MLFLOW_EXPERIMENT_NAME}'")

# Wczytanie danych
print("\nWczytywanie danych...")
df_main = pd.read_csv('data.csv', sep=',')
df_simc = pd.read_csv('Simc.csv', sep=',')
df_terc = pd.read_csv('Terc.csv', sep=',')
df_ulic = pd.read_csv('Ulic.csv', sep=',')
print("Wszystkie pliki wczytane pomyślnie.")

MLflow ustawiony. Eksperyment: 'Warsaw_Street_Prediction_v3_FINAL'

Wczytywanie danych...
Wszystkie pliki wczytane pomyślnie.


In [2]:
# === SEKCJA 2: TWORZENIE KOMPLETNEJ TABELI GEOGRAFICZNEJ ===

# --- Krok 2a: Czyszczenie i przygotowanie tabel TERYT ---
# Upewniamy się, że kluczowe kolumny są w poprawnym formacie (stringi o stałej długości)
df_terc['WOJ'] = df_terc['VoivodeshipNumber'].astype(str).str.zfill(2)
df_terc['POW'] = df_terc['CountyNumber'].astype(str).str.zfill(2)
df_terc['GMI'] = df_terc['CommunityNumber'].astype(str).str.zfill(2)
df_terc['RODZ'] = df_terc['KindNumber'].astype(str)
df_terc['TERYT_GMI'] = df_terc['WOJ'] + df_terc['POW'] + df_terc['GMI'] + df_terc['RODZ']

df_simc['WOJ'] = df_simc['VoivodeshipNumber'].astype(str).str.zfill(2)
df_simc['POW'] = df_simc['CountyNumber'].astype(str).str.zfill(2)
df_simc['GMI'] = df_simc['CommunityNumber'].astype(str).str.zfill(2)
df_simc['RODZ_GMI'] = df_simc['KindNumber'].astype(str)
df_simc['TERYT_MIASTA'] = df_simc['WOJ'] + df_simc['POW'] + df_simc['GMI'] + df_simc['RODZ_GMI']

df_ulic['SYM_UL'] = df_ulic['SymUl'].astype(str).str.zfill(5)
df_ulic['SYM'] = df_ulic['Sym'].astype(str).str.zfill(7)
df_ulic.rename(columns={'Name': 'NAZWA_ULICY', 'Feature': 'CECHA_ULICY'}, inplace=True)

# --- Krok 2b: Łączenie tabel pomocniczych ---
# Łączymy ulice z miejscowościami po symbolu miejscowości 'SYM'
df_geo = pd.merge(df_ulic, df_simc[['Sym', 'Name']], on='Sym', how='left')
df_geo.rename(columns={'Name': 'NAZWA_MIEJSCOWOSCI'}, inplace=True)

print(f"Połączono słowniki. Rozmiar finalnej tabeli geograficznej: {df_geo.shape}")
print("\nPrzykładowe dane z połączonej tabeli geograficznej:")
display(df_geo[['SYM_UL', 'NAZWA_ULICY', 'NAZWA_MIEJSCOWOSCI']].head())

Połączono słowniki. Rozmiar finalnej tabeli geograficznej: (293077, 12)

Przykładowe dane z połączonej tabeli geograficznej:


,SYM_UL,NAZWA_ULICY,NAZWA_MIEJSCOWOSCI
0,25133,Wyzwolenia,Miedźno
1,20298,Słowicza,Wola aleksandra
2,04212,Droga dzików,Jesówka
3,19830,Sienkiewicza,Gwoździany
4,60067,4 czerwca 1989 r.,Chrzanów


In [3]:
# === SEKCJA 3: WZBOGACANIE I CZYSZCZENIE DANYCH GŁÓWNYCH ===

# --- Krok 3a: Wzbogacanie danych głównych o nazwy ulic ---
df_enriched = df_main.copy()
# Upewniamy się, że klucz 'StreetNumber' jest w tym samym formacie co 'SYM_UL'
df_enriched['SYM_UL_str'] = df_enriched['StreetNumber'].astype('Int64').astype(str).str.zfill(5)
# Łączymy po kluczu
df_enriched = pd.merge(df_enriched, df_geo[['SYM_UL', 'NAZWA_ULICY', 'NAZWA_MIEJSCOWOSCI']],
                       left_on='SYM_UL_str', right_on='SYM_UL', how='left')

print(f"Liczba wierszy z przypisaną ulicą ze słownika: {df_enriched['NAZWA_ULICY'].notna().sum()}")

# --- Krok 3b: Stworzenie finalnej kolumny celu 'Ulica_clean' ---
def get_final_street(row):
    # Priorytet ma ulica ze słownika TERYT
    if pd.notna(row['NAZWA_ULICY']):
        return str(row['NAZWA_ULICY']).lower()
    
    # Jeśli nie, próbujemy parsować 'Location'
    location = row['Location']
    if isinstance(location, str) and len(location.split(',')) >= 4:
        street = location.split(',')[3].strip()
        street = re.sub(r'^(ul\.|al\.|pl\.)\s*', '', street, flags=re.IGNORECASE).lower()
        if len(street) > 2:
            return street
    return np.nan

df_enriched['Ulica_clean'] = df_enriched.apply(get_final_street, axis=1)

# --- Krok 3c: Czyszczenie ---
df_enriched.dropna(subset=['Ulica_clean', 'Area', 'Price', 'Description'], inplace=True)
df_enriched['BuiltYear'] = pd.to_datetime(df_enriched['BuiltYear'], format='%Y', errors='coerce')
q_low, q_hi = df_enriched["Price"].quantile(0.01), df_enriched["Price"].quantile(0.99)
df_enriched = df_enriched[(df_enriched["Price"] < q_hi) & (df_enriched["Price"] > q_low)]

print(f"\nPo wzbogaceniu i czyszczeniu -> Pozostało wierszy: {len(df_enriched)}, Unikalnych ulic: {df_enriched['Ulica_clean'].nunique()}")

# --- Krok 3d: TF-IDF ---
vectorizer = TfidfVectorizer(max_features=250, ngram_range=(1, 3), min_df=10)
df_enriched.reset_index(drop=True, inplace=True)
tfidf_features = vectorizer.fit_transform(df_enriched['Description'])
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=['des_tfidf_' + name for name in feature_names])
df_final = pd.concat([df_enriched, tfidf_df], axis=1)

print(f"Finalny kształt zbioru gotowego do modelowania: {df_final.shape}")

Liczba wierszy z przypisaną ulicą ze słownika: 10180035

Po wzbogaceniu i czyszczeniu -> Pozostało wierszy: 8749444, Unikalnych ulic: 4120
Finalny kształt zbioru gotowego do modelowania: (8749444, 306)


In [4]:
# === SEKCJA 4: PRZYGOTOWANIE DO MODELOWANIA ===
MIN_SAMPLES_PER_STREET = 15
street_counts = df_final['Ulica_clean'].value_counts()
streets_to_remove = street_counts[street_counts < MIN_SAMPLES_PER_STREET].index
df_model_ready = df_final[~df_final['Ulica_clean'].isin(streets_to_remove)].copy()
print(f"Po odfiltrowaniu rzadkich ulic -> Pozostało wierszy: {len(df_model_ready)}, Unikalnych ulic: {df_model_ready['Ulica_clean'].nunique()}")

data_train = df_model_ready.sample(frac=0.9, random_state=1122)
data_test = df_model_ready.drop(data_train.index)
fold_groups = data_train['Ulica_clean']
print(f"\nPodział danych -> Treningowe: {data_train.shape}, Testowe: {data_test.shape}")

Po odfiltrowaniu rzadkich ulic -> Pozostało wierszy: 8741834, Unikalnych ulic: 2501

Podział danych -> Treningowe: (7867651, 306), Testowe: (874183, 306)


In [5]:
# === SEKCJA 5: SETUP I TRENING ===
if mlflow.active_run(): mlflow.end_run()

with mlflow.start_run(run_name="Warsaw_Street_Prediction_v3") as run:
    # Użyjemy zautomatyzowanego wyboru cech przez PyCaret, ignorując tylko to, co konieczne
    s = setup(
        data=data_train, test_data=data_test, target='Ulica_clean',
        session_id=1122, n_jobs=4, fold_strategy='groupkfold', fold_groups=fold_groups,
        ignore_features=['SaleId', 'OriginalId', 'Location', 'Description', 'Title', 'Link', 'Phone', 'MainImage', 'OtherImages', 'EncryptedId', 'SYM_UL_str', 'SYM_UL', 'NAZWA_ULICY', 'NAZWA_MIEJSCOWOSCI'],
        log_experiment=False, html=False, verbose=True
    )
    
    print("\nRozpoczynam porównywanie modeli...")
    best_model = compare_models(include=['lightgbm', 'rf', 'et'], sort='F1') # Usunięto xgboost na razie, aby przyspieszyć test

2025/06/18 22:28:43 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



                    Description  \
0                    Session id   
1                        Target   
2                   Target type   
3                Target mapping   
4           Original data shape   
5        Transformed data shape   
6   Transformed train set shape   
7    Transformed test set shape   
8               Ignore features   
9              Numeric features   
10                Date features   
11         Categorical features   
12     Rows with missing values   
13                   Preprocess   
14              Imputation type   
15           Numeric imputation   
16       Categorical imputation   
17     Maximum one-hot encoding   
18              Encoding method   
19               Fold Generator   
20                  Fold Number   
21                     CPU Jobs   
22                      Use GPU   
23               Log Experiment   
24              Experiment Name   
25                          USI   

                                                Value 

In [6]:
# === SEKCJA 6: TRENING I LOGOWANIE DO MLFLOW (ULEPSZONE) ===

if mlflow.active_run():
    mlflow.end_run()

with mlflow.start_run(run_name="Warsaw_Street_Prediction_Run_v2") as run:
    print(f"Rozpoczęto run w MLflow: {run.info.run_id}")
    print("Rozpoczynam porównywanie modeli (ze stabilizacją i modelem dummy)...")
    
    # Dodajemy 'dummy' do listy, aby mieć punkt odniesienia
    models_to_try = ['lightgbm', 'xgboost', 'rf', 'dummy']
    
    best_model = compare_models(include=models_to_try, sort='F1')
    
    print("\nPorównanie modeli zakończone.")
    
    all_models_metrics = pull()
    display(all_models_metrics)
    
    if not all_models_metrics.empty:
        all_models_metrics.to_csv("compare_models_results_v2.csv")
        mlflow.log_artifact("compare_models_results_v2.csv")
        print(f"\nNajlepszy zidentyfikowany model: {best_model}")
    else:
        print("\nUWAGA: Żaden z testowanych modeli nie zakończył treningu pomyślnie.")

Rozpoczęto run w MLflow: d136b02823334da1bc9c15c55c8ff6de
Rozpoczynam porównywanie modeli (ze stabilizacją i modelem dummy)...



Porównanie modeli zakończone.


""



UWAGA: Żaden z testowanych modeli nie zakończył treningu pomyślnie.
